## **1. Téléchargement des données de test**

In [1]:
!wget https://nextcloud.ig.umons.ac.be/s/rYnCgdSmmRerJRa/download/TestData.zip
!unzip TestData.zip -d test_data

--2024-11-23 15:38:56--  https://nextcloud.ig.umons.ac.be/s/rYnCgdSmmRerJRa/download/TestData.zip
Resolving nextcloud.ig.umons.ac.be (nextcloud.ig.umons.ac.be)... 193.190.209.220
Connecting to nextcloud.ig.umons.ac.be (nextcloud.ig.umons.ac.be)|193.190.209.220|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 433130624 (413M) [application/zip]
Saving to: ‘TestData.zip’

TestData.zip        100%[===================>] 413.07M  18.2MB/s    in 32s     

2024-11-23 15:39:30 (12.8 MB/s) - ‘TestData.zip’ saved [433130624/433130624]

Archive:  TestData.zip
   creating: test_data/TestData/
  inflating: test_data/TestData/43.jpg  
  inflating: test_data/TestData/46.jpg  
  inflating: test_data/TestData/69.jpg  
  inflating: test_data/TestData/77.jpg  
  inflating: test_data/TestData/80.jpg  
  inflating: test_data/TestData/85.jpg  
  inflating: test_data/TestData/86.jpg  
  inflating: test_data/TestData/91.jpg  
  inflating: test_data/TestData/94.jpg  
  inflating: test_d

## **2. Importation des librairies**

In [2]:
import torch
from PIL import Image
from pathlib import Path
import pandas as pd
from torchvision import transforms
from tqdm import tqdm
import os
from torch.utils.data import DataLoader
import torchvision.datasets as datasets

# **3. Téléchargement de la BD de test**

In [3]:
img_size = 224
folder_path  = 'test_data/TestData'

model = torch.jit.load("model_jit.pth")

In [4]:
test_transform = transforms.Compose([
        transforms.Resize((img_size, img_size)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
    ])

## **4. Générer le fichier CSV de prédictions pour soumission au challenge Kaggle**

In [5]:
def predict_folder(model, folder_path):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)
    model.eval()


    predictions = []
    filenames = []

    for img_path in os.listdir(folder_path):
        with torch.no_grad():
            # Load and preprocess image
            image = Image.open(folder_path+'/'+img_path).convert('RGB')
            image = test_transform(image).unsqueeze(0).to(device)

            # Get prediction
            output = model(image)
            #main_output = output[0]
            #_, predicted = main_output.max(1)
            _, predicted = output.max(1)

            predictions.append(predicted.item())
            filenames.append(img_path)

    # Create submission dataframe
    submission_df = pd.DataFrame({
        'ID': filenames,
        'Label': predictions
    })

    # Save to CSV
    submission_df.to_csv('submission.csv', index=False)
    return submission_df


submission = predict_folder(model, folder_path)